In [1]:
# default_exp learn


In [2]:
# export

from peptide.imports import *
from peptide.utils import *
from peptide.data import ProteinDataset


In [3]:
# hide

from nbdev.showdoc import *


# Learn
> Learner class for supervised, unsupervised and semi supervised learning with Protein data.

#hide

- To suppress warning in multithreaded runs - https://github.com/scikit-learn/scikit-learn/issues/12939
    - But makes it slow
- To revert - delete the following
```python
from sklearn.utils import parallel_backend
..
    with parallel_backend("multiprocessing"):

```

In [4]:
# export

# from sklearn.utils import parallel_backend


class Learner:
    """Class for training and prediction."""

    def __init__(
        self,
        X_train: np.ndarray,
        y_train: np.ndarray,
        X_test: np.ndarray,
        y_test: np.ndarray,
        ohe: bool = False,
        scaler: bool = False,
        pca: bool = True,
        pca_n_components: int = 50,
        param_grids: list = None,
    ):
        """Initialize learner for training and prediction."""
        self.classifiers = ["LogisticRegression", "LinearSVC", "XGBClassifier"]
        self.X_train, self.y_train = X_train, y_train
        self.X_test, self.y_test = X_test, y_test
        self.ohe, self.scaler, self.pca = ohe, scaler, pca
        self.pca_n_components = pca_n_components
        self.pipeline = self.create_pipeline()
        self.param_grids = (
            get_default_param_grid() if param_grids is None else param_grids
        )

        self.grid_list, self.train_results = [], []
        self.predict_results = None

    def create_pipeline(self) -> Pipeline:
        """Create and return pipeline"""

        steps = []
        if self.ohe:
            steps.append(("ohe", OneHotEncoder(handle_unknown="ignore", sparse=False)))
        if self.scaler:
            steps.append(("scaler", StandardScaler()))
        if self.pca:
            steps.append(("pca", PCA(n_components=self.pca_n_components)))
        steps.append(("classifier", "passthrough"))

        pipe = Pipeline(steps)

        return pipe

    def train(
        self, scoring: str = "accuracy", cv: int = 3, n_jobs: int = -1
    ) -> tuple[list, list]:
        """Run GridSearchCV for all models on X_train and y_train of dataset.

        Args:
            scoring: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
            cv: defaults to 3-fold cv
            n_jobs: defaults to -1 to use all cores
        Returns:
            train_results: list of grid search results
            grid_list: list of trained grid objects
        """

        result_list = []
        grid_list = []

        for classifier, param_grid in zip(self.classifiers, self.param_grids):
            print(f"Starting grid search for {classifier}")
            grid = GridSearchCV(
                estimator=self.pipeline,
                param_grid=param_grid,
                n_jobs=n_jobs,
                cv=cv,
                scoring=scoring,
                verbose=1,
            )

            # with parallel_backend("multiprocessing"):
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")
                grid.fit(self.X_train, self.y_train)

            result_list.append(pd.DataFrame.from_dict(grid.cv_results_))
            grid_list.append(grid)

        self.train_results = result_list
        self.grid_list = grid_list

        return self.train_results, self.grid_list

    def get_top_5_train_results(self) -> list:
        "Return top 5 results for each grid"
        results = []
        for result in self.train_results:
            results.append(result.sort_values("rank_test_score")[:5])
        return results

    def predict(self) -> pd.DataFrame:
        """Get predictions on the datasets X_test from best estimators of GridSearchCV."""
        results = []
        for classifier, grid in zip(self.classifiers, self.grid_list):
            preds = grid.predict(self.X_test)
            result = [
                classifier,
                grid.best_params_,
                accuracy_score(self.y_test, preds),
                recall_score(self.y_test, preds),
                precision_score(self.y_test, preds),
                f1_score(self.y_test, preds),
            ]
            results.append(result)

        self.predict_results = pd.DataFrame(
            results,
            columns=[
                "classifier",
                "best_params",
                "accuracy",
                "recall",
                "precision",
                "f1",
            ],
        )
        return self.predict_results

    ## Unsupervised Learning ##

    def pick_k(
        self,
        max_clusters: int = 10,
        pca_n_components: int = 50,
    ) -> np.ndarray:
        """Visualize elbow & silhouette plots and print silhouette scores to help determine k for KMeans."""

        # concat X
        X = np.concatenate((self.X_train, self.X_test), axis=0)
        assert (self.X_train.shape[0] + self.X_test.shape[0]) == X.shape[0]

        if self.ohe:  # One Hot Encode X
            ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
            X = ohe.fit_transform(X)

        if self.scaler:  # Scale
            scalr = StandardScaler()
            X = scalr.fit_transform(X)

        if self.pca:  # Dim Reduce X
            pca = PCA(n_components=pca_n_components)
            X = pca.fit_transform(X)

        # visualize elbow plot
        visualize_elbow(X, np.arange(2, max_clusters))
        # visualize silhouette scores and plot
        plot_silhouette_scores(max_clusters=max_clusters, X=X)

        return X

    def analyze_clusters(
        self, X_pca: np.ndarray, k: int, random_state: int = 10
    ) -> None:
        """Analyze and plot clusters identified by KMeans."""

        km = KMeans(n_clusters=k, random_state=random_state).fit(X_pca)
        print(f"Cluster counts: {Counter(km.labels_)}")
        visualize_clusters(km.labels_, X_pca)

    ## Semi Supervised Learning ##

    def run_label_spreading(
        self,
        pca_n_components: int = 50,
    ) -> None:
        """Run Lanel Spreading and print classification report."""

        # concat X
        X = np.concatenate((self.X_train, self.X_test), axis=0)
        assert (self.X_train.shape[0] + self.X_test.shape[0]) == X.shape[0]

        # concat y
        y = np.concatenate((self.y_train, np.full(self.y_test.shape, -1)), axis=0)
        assert (self.y_train.shape[0] + self.y_test.shape[0]) == y.shape[0]

        if self.ohe:  # One Hot Encode X
            ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
            X = ohe.fit_transform(X)

        if self.scaler:  # Scale
            scalr = StandardScaler()
            X = scalr.fit_transform(X)

        if self.pca:  # Dim Reduce X
            pca = PCA(n_components=pca_n_components)
            X = pca.fit_transform(X)

        # Run LableSpreading
        lbl_spread = LabelSpreading(kernel="knn", alpha=0.01)
        lbl_spread.fit(X, y)
        semi_sup_preds = lbl_spread.transduction_[self.X_train.shape[0] :]
        assert semi_sup_preds.shape[0] == self.X_test.shape[0]

        # print result
        print(classification_report(self.y_test, semi_sup_preds))


## Export -

In [7]:
# hide
from nbdev.export import *

notebook2script()


Converted 00_basics.ipynb.
Converted 01_data.ipynb.
Converted 02_learn.ipynb.
Converted 03_onehot.ipynb.
Converted 04_lstm.ipynb.
Converted 05_transformer.ipynb.
Converted 99_utils.ipynb.
Converted index.ipynb.
